In [ ]:
#conda install -c conda-forge pyarrow xgboost mlflow

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
CACHED_DATA_DIR = "/goalimpacct/spark_data_cache/result_parquet"
resultDf =  pd.read_parquet(CACHED_DATA_DIR)


In [ ]:
CACHED_PLAYER_TIME_DIR = "/goalimpacct/spark_data_cache/player_time_parquet"
ptDf =  pd.read_parquet(CACHED_PLAYER_TIME_DIR)
ptDf.drop(columns=[':team', ':tournament', ':saison', ':date', ':in-time',':out-time'], inplace=True)

In [ ]:
def getXMatchesAttributes(xList):
    array = []
    
    for i in xList:
        array.append(':playtimeLast' + str(i) + 'Matches')
        array.append(':totalOffPointsLast' + str(i) + 'Matches')
        array.append(':totalDefPointsLast' + str(i) + 'Matches')
        array.append(':avgOffPointsLast' + str(i) + 'Matches')
        array.append(':avgDefPointsLast' + str(i) + 'Matches')
        array.append(':totalRankedOffPointsLast' + str(i) + 'Matches')
        array.append(':totalRankedDefPointsLast' + str(i) + 'Matches')
        array.append(':avgRankedOffPointsLast' + str(i) + 'Matches')
        array.append(':avgRankedDefPointsLast' + str(i) + 'Matches')
    
    return array

        
#xMatchAttribures = getXMatchesAttributes([1, 2, 3, 4, 5, 6, 8 , 13, 21])
xMatchAttribures = getXMatchesAttributes([1, 2, 3, 4, 5, 6, 8 , 13, 21])


In [ ]:
MATCH_CSV = "/goalimpacct/data_compressed/match/compressed.csv"
matchDf = pd.read_csv(MATCH_CSV)

In [ ]:
GOAL_CSV = "/goalimpacct/data_compressed/goal/compressed.csv"
goalDf = pd.read_csv(GOAL_CSV)

In [ ]:
SUBSTITUTION_CSV = "/goalimpacct/data_compressed/substitution/compressed.csv"
substitutionDf = pd.read_csv(SUBSTITUTION_CSV)

In [ ]:
PLAYER_CSV = "/goalimpacct/data_compressed/player/compressed.csv"
playerDf = pd.read_csv(PLAYER_CSV)


In [ ]:
matchDf.rename(columns={ ":id" : ":match"}, inplace=True)


In [ ]:
matchesGoals = pd.merge(matchDf, goalDf, how='inner', on = ':match')
#matchesGoals

In [ ]:
matchesHomeGoals = matchesGoals[(matchesGoals[":team"] == matchesGoals[":home"])]
matchesHomeGoals.loc[:, ':home?'] = True
matchesHomeGoals.rename(columns={ ":guest" : ":opponent"}, inplace=True)
matchesHomeGoals.drop(columns=[':home'], inplace=True)


In [ ]:
matchesGuestGoals = matchesGoals[(matchesGoals[":team"] != matchesGoals[":home"])]
matchesGuestGoals.loc[:, ':home?'] = False
matchesGuestGoals.rename(columns={ ":home" : ":opponent"}, inplace=True)
matchesGuestGoals.drop(columns=[':guest'], inplace=True)


In [ ]:
matchesLocGoals = pd.concat([matchesHomeGoals, matchesGuestGoals])


In [ ]:
goalCount = matchesLocGoals.groupby([':match', ':date', ':saison', ':team', ':opponent', ':home?', ':match-length']).size().reset_index(name='goals')


In [ ]:
cleanMatch = matchDf[[':match', ':date', ':saison', ':guest', ':home', ':match-length']]

cleanHomeMatch = cleanMatch.rename(columns={ ":home" : ":team", ":guest": ":opponent"})
cleanHomeMatch[':home?'] = True
cleanHomeMatch['goals'] = 0

cleanGuestMatch = cleanMatch.rename(columns={ ":guest" : ":team", ":home": ":opponent"})
cleanGuestMatch[':home?'] = False
cleanGuestMatch['goals'] = 0

zeroGoalCount = pd.concat([cleanHomeMatch[[':match', ':date', ':saison', ':team', ':opponent', ':home?', 'goals', ':match-length']], 
                           cleanGuestMatch[[':match', ':date', ':saison', ':team', ':opponent', ':home?', 'goals', ':match-length']]])


In [ ]:
allGoalCount = pd.concat([goalCount, zeroGoalCount])
goalCountDf = allGoalCount.groupby([':match', ':date', ':saison', ':team', ':opponent', ':home?', ':match-length']
                                   , sort=False)['goals'].max().reset_index(name='goals')


In [ ]:
playerDf = playerDf.astype(str)


In [ ]:
dateOfMatch = matchDf[[':match', ':date']]

dateOfMatch.loc[:, ':match'] = dateOfMatch[':match'].astype(str)
dateOfMatch.loc[:, ':date'] = pd.to_datetime(dateOfMatch[':date'],errors='coerce')


In [ ]:
dateOfMatchFiltered = dateOfMatch[dateOfMatch[':date'].dt.year >= 2014]
dateOfMatchFiltered.shape

In [ ]:
years = pd.DataFrame(dateOfMatchFiltered[':date'].dt.year)
pd.DataFrame(years[':date'].value_counts()).plot.pie(subplots=True)

In [ ]:
playerAndDateDF = pd.merge( dateOfMatchFiltered, playerDf, how='left', left_on=[':match'], right_on = [':match'])

In [ ]:
rawMerge = pd.merge(playerAndDateDF, resultDf,  how='left', 
                    left_on=[':team', ':player'], right_on = [':team',':player'])

In [ ]:
rawMerge.shape

In [ ]:
filtered = rawMerge[rawMerge[':date'] > rawMerge[':target-match-timestamp']]
filtered = rawMerge[rawMerge[':match_x'] != rawMerge[':match_y']]

In [ ]:
filtered.shape

In [ ]:
maxDate = filtered.groupby([':match_x', ':date', ':team', 
                            ':player'])[':target-match-timestamp'].max().reset_index(name=':target-match-timestamp')

In [ ]:
playersToMatch = pd.merge(maxDate, resultDf,  how='left', 
                          left_on=[':team', ':player', ':target-match-timestamp'], 
                          right_on = [':team', ':player', ':target-match-timestamp'])

playersToMatch.drop(columns=[':match', ':tournament', ':target-match-timestamp', ':saison', ':date'], inplace=True)
playersToMatch.rename(columns={ ":match_x" : ":match"}, inplace=True)



In [ ]:
playersWithPtToMatch = pd.merge(playersToMatch, ptDf,  how='left', left_on=[':match', ':player'], right_on = [':match', ':player'])

for attr in xMatchAttribures:   
    playersWithPtToMatch.loc[:, attr] = playersWithPtToMatch[attr] * playersWithPtToMatch[':playtime']
        
    

In [ ]:
teamToMatch = playersWithPtToMatch.groupby([':match', ':team']).sum().reset_index()

In [ ]:
# create oppenent team df
opponentTeamToMatch = teamToMatch.add_prefix("opponent_")


In [ ]:
# join team data

goalCountDf.loc[:, ':match'] = goalCountDf[':match'].astype(int)
goalCountDf.loc[:, ':team'] = pd.to_numeric(goalCountDf[':team'])
teamToMatch.loc[:, ':match'] = teamToMatch[':match'].astype(int)
teamToMatch.loc[:, ':team'] = pd.to_numeric(teamToMatch[':team'])

goalTeamDf = pd.merge(goalCountDf, teamToMatch, how='inner', on=[':match', ':team'])


In [ ]:
# join opponent data

opponentTeamToMatch.loc[:, 'opponent_:match'] = opponentTeamToMatch['opponent_:match'].astype(int)
opponentTeamToMatch.loc[:, 'opponent_:team'] = pd.to_numeric(opponentTeamToMatch['opponent_:team'])

goalTeamDf = pd.merge(goalTeamDf, opponentTeamToMatch, how='inner', left_on=[':match', ':opponent'], right_on=['opponent_:match', 'opponent_:team'])


In [ ]:
# add date stuff
goalTeamDf.loc[:, ':date'] = pd.to_datetime(goalTeamDf[':date'])

goalTeamDf.loc[:, 'Year'] = goalTeamDf[':date'].dt.year 
goalTeamDf.loc[:, 'Month'] = goalTeamDf[':date'].dt.month
goalTeamDf.loc[:, 'Day'] = goalTeamDf[':date'].dt.day

In [ ]:
goalTeamDf

In [ ]:
# remove useless columns

trainDF = goalTeamDf.drop(columns=[':date', ':team', ':opponent', ':match'])
trainDF = trainDF.dropna()

for attr in xMatchAttribures:   
    trainDF.loc[:, attr] = trainDF[attr] / trainDF[':match-length']
    trainDF.loc[:, 'opponent_' + attr] = trainDF['opponent_' + attr] / trainDF[':match-length']

In [ ]:
trainDF

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from xgboost import XGBRegressor
import mlflow
import mlflow.sklearn

import time

import os
# otherwise xgboost will kill the kernel
os.environ['KMP_DUPLICATE_LIB_OK']='True'

mlflow.set_tracking_uri('http://127.0.0.1:5000')

In [ ]:
loopDF = trainDF.head(1000)

In [ ]:
X = loopDF.drop(columns=['goals'])
y = loopDF.goals.astype(float)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

imputer = SimpleImputer()
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

print('There are {} samples in the training set and {} samples in the test set'.format(
X_train.shape[0], X_test.shape[0]))

In [ ]:
# XGBOOST MODEL
n_estimators = 500
learning_rate = 0.1
early_stopping_rounds = 40


In [ ]:
def calc_model(expID, n_estimators, learning_rate, early_stopping_rounds):

    mlflow.set_experiment(str(expID))
    
    with mlflow.start_run():
        

        model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate)
        model.fit(X_train, y_train, early_stopping_rounds=early_stopping_rounds, eval_set=[(X_test, y_test)]
                  , verbose=False)

        predictions = model.predict(X_test)
        predictions_rounded = predictions.round(0)

        mae = mean_absolute_error(predictions_rounded, y_test)
        mse = mean_squared_error(predictions_rounded, y_test)
        r2 = r2_score(predictions_rounded, y_test)
        print("xgboost:mean_absolute_error={}".format(mae))
        print("xgboost:mean_squared_error={}".format(mse))
        print("xgboost:r2_score={}".format(r2))


        mlflow.log_param("training_size", len(X_train))
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("early_stopping_rounds", early_stopping_rounds)
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(model, "model")

In [ ]:
start = time.time()

for n_estimators in [100, 500, 1000, 5000]:
    for learning_rate in np.arange(0.1, 1.0, 0.1):
        calc_model(start, n_estimators, learning_rate, early_stopping_rounds)
    
end = time.time()    
print("finished in "+ str(end - start))



In [ ]:

X = trainDF.drop(columns=['goals'])
y = trainDF.goals.astype(float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

imputer = SimpleImputer()
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate)
model.fit(X_train, y_train, early_stopping_rounds=early_stopping_rounds, eval_set=[(X_test, y_test)]
                  , verbose=False)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions_rounded = predictions.round(0)

In [ ]:
mae = mean_absolute_error(predictions, y_test)
mae

In [ ]:
mse = mean_squared_error(predictions, y_test)
mse

In [ ]:
diffDF = pd.DataFrame({'pred':predictions, 'actual': y_test}).sort_index()

In [ ]:
diffDF

In [ ]:
diffDF.plot()